In [14]:
import argparse
import gzip
import os 
import sys
import time

import numpy as np
import urllib
#from six.moves import xrange
import tensorflow  as tf

In [15]:
source_url="https://storage.googleapis.com/cvdf-datasets/mnist/"
work_directory="data_mnist"
image_size=28
num_channel=1
pixel_depth=255
num_labels=10
validation_size=5000
seed=423
batch_size=128
num_epochs=10
eval_batch_size=64
eval_frequency=10
n_classes=10


FLAGS=None

In [16]:
def data_type():
    if FLAGS.use_fps16:
        return tf.float16
    else:
        return tf.float32
    
def maybe_download(filename):
    if not tf.gfile.Exists(work_directory):
        tf.gfile.MakeDirs(work_directory)
    filepath=os.path.join(work_directory,filename)
    if not tf.gfile.Exists(filepath):
        filepath,_=urllib.request.urlretrieve(source_url+filename,filepath)
        with tf.gfile.GFile(filepath) as f:
            size=f.size()
        print("successfully download",filename,size,"bytes.")
    return filepath

def extract_data(filename,num_images):
    
    print("extracting",filename)
    with gzip.open(filename) as byt:
        byt.read(16)
        buf=byt.read(image_size*image_size*num_images*num_channel)
        data=np.frombuffer(buf,dtype=np.uint8).astype(np.float32)
        data=(data-(pixel_depth/2.0))/pixel_depth
        data=data.reshape(num_images,image_size,image_size,num_channel)
    return data
    
def extract_label(filename,num_images):
    print("extracting",filename)
    with gzip.open(filename) as byt:
        byt.read(8)
        buf=byt.read(1*num_images)
        labels=np.frombuffer(buf,dtype=np.uint8).astype(np.int64)
    return labels

In [17]:
# get data
train_data_filename=maybe_download("train-images-idx3-ubyte.gz")
train_labels_filename=maybe_download("train-labels-idx1-ubyte.gz")
test_data_filename=maybe_download("t10k-images-idx3-ubyte.gz")
test_labels_filename=maybe_download("t10k-labels-idx1-ubyte.gz")

# extract data
train_data=extract_data(train_data_filename,60000)
train_labels=extract_label(train_labels_filename,60000)
test_data=extract_data(test_data_filename,10000)
test_labels=extract_label(test_labels_filename,10000)

# generate validation data
validation_data=train_data[:validation_size]
validation_labels=train_labels[:validation_size]
train_data=train_data[validation_size:]
train_labels=train_labels[validation_size:]



extracting data_mnist/train-images-idx3-ubyte.gz
extracting data_mnist/train-labels-idx1-ubyte.gz
extracting data_mnist/t10k-images-idx3-ubyte.gz
extracting data_mnist/t10k-labels-idx1-ubyte.gz


In [18]:
tf.reset_default_graph()
train=True
train_size=train_labels.shape[0]
epochs=num_epochs
epsilon = 1e-3
#phase=True
#
x=tf.placeholder(dtype=tf.float32,shape=[None,image_size,image_size,num_channel],
                name="x")
y=tf.placeholder(dtype=tf.int32,shape=[None],name="y")
phase = tf.placeholder(tf.bool, name='phase')


#CONV LAYER_1
conv1_weights=tf.Variable(tf.truncated_normal([5,5,num_channel,32],stddev=0.5),name="conv1_w")
conv1_bias=tf.Variable(tf.truncated_normal([32],0,0.1),name="conv1_b")
conv1=tf.nn.bias_add(tf.nn.conv2d(x,conv1_weights,strides=[1,1,1,1],padding="SAME",name="conv1"),conv1_bias)
conv1_bn=tf.contrib.layers.batch_norm(conv1,center=True,scale=True,is_training=phase)
relu1=tf.nn.relu(conv1_bn,name="relu1")
pool1=tf.nn.max_pool(relu1,ksize=(1,2,2,1),strides=[1,2,2,1],padding="VALID",name="pool1")

#CONV LAYER_2
conv2_weigts=tf.Variable(tf.truncated_normal([3,3,32,64],stddev=0.1),name="conv2_w")
conv2_bias=tf.Variable(tf.truncated_normal([64],0,0.1),name="conv2_b")
conv2=tf.add(tf.nn.conv2d(pool1,conv2_weigts,strides=[1,1,1,1],padding="SAME",name="conv2"),conv2_bias)
conv2_bn=tf.contrib.layers.batch_norm(conv2,center=True,scale=True,is_training=phase)
relu2=tf.nn.relu(conv2_bn,name="relu2")
pool2=tf.nn.max_pool(relu2,ksize=(1,2,2,1),strides=[1,2,2,1],padding="VALID",name="pool2")

#FULL_CONNECTED LAYER_3
reshape=tf.reshape(pool2,shape=[-1,pool2.shape[1]*pool2.shape[2]*pool2.shape[3]])
fc1_weights=tf.Variable(tf.random_normal([int(reshape.shape[1]),512],0,0.5),name="fc1_w")
fc1_bias=tf.Variable(tf.random_normal([512],0,0.1),name="fc1_b")

fc1=tf.matmul(reshape,fc1_weights)+fc1_bias
fc1=tf.nn.relu(fc1,name="fc1_relu")

if train:
    fc1=tf.nn.dropout(fc1,0.9,seed=seed,name="fc1_dropout")
    
#FULL_CONNECTED LAYER_4
fc2_weights=tf.Variable(tf.random_normal([512,n_classes],0,0.5),name="fc2_w")
fc2_bias=tf.Variable(tf.random_normal([n_classes],0,0.1),name="fc2_b")
logits=tf.add(tf.matmul(fc1,fc2_weights),fc2_bias,name="logits")
y_proba=tf.nn.softmax(logits,name="y_proba")
    
       

In [19]:
with tf.name_scope("train"):
    loss1=tf.losses.sparse_softmax_cross_entropy(y,logits)
    loss2=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits))
    
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        # Ensures that we execute the update_ops before performing the train_step
        train_op=tf.train.AdamOptimizer().minimize(loss1)                                                                   
                

In [20]:
with tf.name_scope("eval"):
    correct=tf.nn.in_top_k(logits,y,1)
    accuracy=tf.reduce_mean(tf.cast(correct,dtype=tf.float32))

In [21]:
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [22]:
def batch_generate(data,labels,batch_size):
    global i
    batch_x=data[i:i+batch_size]
    batch_y=labels[i:i+batch_size]
    i+=batch_size
    #print("index :",int(i/batch_size))
    return batch_x,batch_y

def station_epoch(step,total_steps): 
    dot_number=int(step/total_steps*100/2)
    #print(step,total_steps)
    #print(dot_number)
    sta="."*dot_number
    null=" "*(50-dot_number)
    print("   ",sta+null,"% {}".format(int((step+1)/total_steps*100)),end='\r')


In [23]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        i=0
        print("# ---------------------------------------------------------- #")
        total_steps=int(validation_data.shape[0]/batch_size)
        for step in range(total_steps):
            #print(step)
            batch_x,batch_y=batch_generate(validation_data,validation_labels,batch_size)
            sess.run(train_op,feed_dict={x:batch_x,y:batch_y,phase:1})
            station_epoch(step,total_steps)
        
        print(end="\n")
        #print("Calculation Loss ...")    
        l,a=sess.run([loss1,accuracy],feed_dict={x:test_data,y:test_labels,phase:0})
        print("    ","Epoch :",epoch,"| Accuracy : {:.4f} | Loss : {:.4f}".format(a,l))

# ---------------------------------------------------------- #
    ................................................   % 100
     Epoch : 0 | Accuracy : 0.3371 | Loss : 145.9312
# ---------------------------------------------------------- #
    ................................................   % 100
     Epoch : 1 | Accuracy : 0.3737 | Loss : 124.9279
# ---------------------------------------------------------- #
    ................................................   % 100
     Epoch : 2 | Accuracy : 0.3708 | Loss : 115.5912
# ---------------------------------------------------------- #
    ................................................   % 100
     Epoch : 3 | Accuracy : 0.4722 | Loss : 86.6411
# ---------------------------------------------------------- #
    ................................................   % 100
     Epoch : 4 | Accuracy : 0.4781 | Loss : 72.7551
# ---------------------------------------------------------- #
    ................................................  